## Web Search with Foundry Agent

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 
%pip install openai==1.109.1 
%pip install python-dotenv
%pip install azure-identity

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, WebSearchPreviewTool, ApproximateLocation

load_dotenv(dotenv_path=os.path.join(os.pardir, ".env"))

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

### Setting Up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client()

### Create an Agent with the Web Search Tool

In [ ]:
agent = project_client.agents.create_version(
    agent_name="web-search-agent",
    definition=PromptAgentDefinition(
        model = model_deployment_name,
        instructions="You are a helpful assistant that uses web search to answer user queries.",
        tools = [
            WebSearchPreviewTool()
        ]
    )
)

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Chat With the Agent

In [ ]:
user_query = "What are the latest advancements in india on AI ?"

In [ ]:
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body={
        "agent": {
            "name": "web-search-agent",
            "type": "agent_reference"
        }
    },
    input = user_query
)

print(f"Agent response: {response.output_text}")